Import cell

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegressionCV, LogisticRegression, RidgeClassifier, \
    RidgeClassifierCV, Ridge
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from pandas import MultiIndex, Int16Dtype
from statistics import mean
import math 
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# In the cell below i run a visilastion_graphe to see how my features look.

In [3]:
def visilastion_graphe(X, Y):
    fig, axs = plt.subplots(math.ceil((len(X.columns) / 5)), 5, figsize=(15, 15))
    fig.tight_layout()
    count = 0
    for col in X.columns:
        axs[int(count / 5), count % 5].grid()
        axs[int(count / 5), count % 5].set_title(col, color = 'red')
        axs[int(count / 5), count % 5].hist(X[col], bins=50)
        count += 1
#     axs[int(count / 5), count % 5].grid()
#     axs[int(count / 5), count % 5].hist(Y, bins=50)
#     axs[int(count / 5), count % 5].set_title("Likert Raiting", color='red')
    plt.show()

# In the cell below i run a multiplay models and chose a one of them as my main model.

In [24]:
def modelss(X, Y, X_Test, Y_Test):
    models_lr = Ridge()
    models_knn = KNeighborsRegressor(n_neighbors = 4)
    models_cart = DecisionTreeRegressor(max_depth=10,criterion = "absolute_error", random_state=42)
    models_svm = SVR(C=1.0, epsilon=0.2)
    models_xgb = XGBRegressor(tree_method="hist",eval_metric=mean_absolute_error)
    models_randomforest = RandomForestRegressor(max_depth=14, n_estimators=50, random_state=42)
    models = [models_lr, models_knn, models_cart, models_svm, models_xgb, models_randomforest]
    models_name = ['Linear Regression', 'KNN', 'Decision Tree', 'SVM', 'XGB', 'Random Forest', 'Mean - Stupid', 'Common - Stupid']
    scores = []
    predicts = []
    scores_val = []
    strtfdKFold = StratifiedKFold(n_splits = 10)
    kfold = strtfdKFold.split(X, Y)
    for k, (train, test) in enumerate(kfold):
        #X_train, Y_train = over_sample_smote(X.iloc[train, :], Y.iloc[train])
        X_train, Y_train = X.iloc[train, :], Y.iloc[train]
        models = list(map(lambda model: model.fit(X_train, Y_train), models))
        predicts = list(map(lambda model: model.predict(X.iloc[test, :]), models))
        predicts.append(np.ones(len(test))*np.mean(Y.iloc[test]))
        predicts.append(np.ones(len(test))*Y.iloc[test].value_counts().idxmax())
        temp = np.array(predicts)
        temp[temp > 5] = 5
        temp[temp < 1] = 1
        predicts = temp
        score = list(map(lambda predict: np.mean(np.abs(predict - Y.iloc[test])), predicts))
        scores.append(score)
#     models = list(map(lambda model: model.fit(X, Y), models))
#     Test = list(map(lambda model: model.predict(X_Test), models))
#     Test.append(np.ones(len(X_Test)) * predicts[5][1]) #'Mean - Stupid'
#     Test.append(np.ones(len(X_Test)) * predicts[6][1]) #'Common - Stupid'
#     score_test = list(map(lambda Test: np.mean(np.abs(Test - Y_Test)), Test))
#     scores_test.append(score_test)
#     ziped = zip(models_name, np.mean(scores, axis = 0),np.std(scores, axis = 0), scores_test )
    ziped = zip(models_name, np.mean(scores, axis = 0),np.std(scores, axis = 0))
    ziped = sorted(list(ziped), key=lambda x: x[1], reverse=False)
    #print('\nCross-Validation scores:\n %3s' % (ziped))
    return ziped, predicts

# In the cell below i ran a Random forest with differnt depths and then chose one of them by knee rule 

In [5]:
def Random_Forest_desicion_of_depths(X,Y): 
    total_scores = []
    predicts = []
    depths = range(1,26)
    for max_depth in depths:
        strtfdKFold = StratifiedKFold(n_splits = 10)
        kfold = strtfdKFold.split(X, Y)
        scores = []
        for k, (train, test) in enumerate(kfold):
            model = RandomForestRegressor(max_depth = max_depth, criterion = "absolute_error", random_state=42)
            model.fit(X.iloc[train, :], Y.iloc[train])
            predict = model.predict(X.iloc[test, :])
            score =  np.mean(np.abs(predict - Y.iloc[test]))
            scores.append(score)
        total_scores.append(sum(scores)/len(scores))
    plt.plot(depths, total_scores, label="test")
    plt.xlabel('depths')
    plt.ylabel('MAE')
    plt.title('score againts max depth of the tree')
    plt.legend()
    plt.show()
    return total_scores

# In the cell below i ran a Random forest with differnt trees and then chose one of them by knee rule 

In [6]:
def Random_Forest_desicion_of_trees(X,Y):
    total_scores = [] 
    predicts = [] 
    num_of_trees = [1, 10, 25, 50, 100, 200, 400] 
    for n in num_of_trees: 
        strtfdKFold = StratifiedKFold(n_splits = 10)
        kfold = strtfdKFold.split(X, Y)
        scores = [] 
        for k, (train, test) in enumerate(kfold):
            model = RandomForestRegressor(max_depth = 14, n_estimators = n, random_state=42)
            model.fit(X.iloc[train, :], Y.iloc[train]) 
            predict = model.predict(X.iloc[test, :]) 
            score = np.mean(np.abs(predict - Y.iloc[test])) 
            scores.append(score)
        total_scores.append(sum(scores)/len(scores)) 
    plt.plot(num_of_trees, total_scores, label="test")
    plt.xlabel('num of trees')
    plt.ylabel('MAE')
    plt.title('score againts num of trees')
    plt.legend()
    plt.show()
    return total_scores

# In the cell below i ran a Random forest my chosen depth and trees. and return my score and feature importance.

In [7]:
def Random_Forest(X,Y):
    strtfdKFold = StratifiedKFold(n_splits = 10)
    kfold = strtfdKFold.split(X, Y)
    scores = [] 
    feature_importances = []
    fetures = list(X.columns.values)
    for k, (train, test) in enumerate(kfold):
        model = RandomForestRegressor(max_depth = 14, n_estimators = 50, random_state=42)
        model.fit(X.iloc[train, :], Y.iloc[train]) 
        predict = model.predict(X.iloc[test, :]) 
        score = np.mean(np.abs(predict - Y.iloc[test])) 
        scores.append(score)
        feature_importances.append(model.feature_importances_)
    total_feature_importances = np.mean(feature_importances, axis = 0)
    ziped = zip(fetures, total_feature_importances)
    ziped = sorted(list(ziped), key=lambda x: x[1], reverse = True)
    return scores, ziped

# SMOTE

In [8]:
def over_sample_smote(X,Y):
    oversample = SMOTE()
    X_smote, Y_smote = oversample.fit_resample(X, Y)
    plt.scatter(Y, range(len(Y)), label = 'original')
    plt.scatter(Y_smote[len(Y)+1:], range(len(Y)+1,len(Y_smote)) , label = 'resampling with smote')
    plt.legend()
    plt.ylabel('id of sample')
    plt.xlabel('value of sample')
    plt.show()
    return  X_smote, Y_smote

In [9]:
def over_sample_adasyn(X,Y):
    oversample = ADASYN()
    X_smote, Y_smote = oversample.fit_resample(X, Y)
    plt.scatter(Y, range(len(Y)), label = 'original')
    plt.scatter(Y_smote[len(Y)+1:], range(len(Y)+1,len(Y_smote)) , label = 'resampling with smote')
    plt.legend()
    plt.ylabel('id of sample')
    plt.xlabel('value of sample')
    plt.show()
    return  X_smote, Y_smote

# Main

In [66]:
df = pd.read_csv(r"result_final.csv")
df = df.fillna(0)
X = df.drop(['Likert Raiting', 'URL', 'Original row ID'], axis=1)  # features
Y = df['Likert Raiting']  # labels
selector = SelectKBest(chi2, k = 15)
selctor = selector.fit(X, Y)
mask = selector.get_feature_names_out()
X_chi = X[mask]

df1 = pd.read_csv(r"secondary_final.csv")
df1 = df1.fillna(0)
X_Test = df1.drop(['Likert Raiting', 'URL', 'Original row ID', 'edu', 'biz', 'info', 'io','Rater ID'], axis=1)  # features
Y_Test = df1['Likert Raiting']  # labels

# X_smote, Y_smote = over_sample_smote(X,Y)
# X_adasyn, Y_adasyn = over_sample_adasyn(X,Y)

In [57]:
X_chi

,Result Rank,Links,a tags,Word count,Misspelled words,misspelled_percent (%),Char count,Img count,Banner count,Link depth,Special characters,Outdated dates,com,net,edu,biz,gov
0,2.0,3.0,81.0,146.0,39.0,26.71,1341.0,36.0,0.0,2.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0
1,3.0,4.0,174.0,461.0,82.0,17.79,4256.0,70.0,0.0,2.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4.0,4.0,49.0,737.0,93.0,12.62,5527.0,9.0,0.0,3.0,12.0,0.0,1.0,0.0,0.0,0.0,0.0
3,5.0,7.0,247.0,921.0,235.0,25.52,8434.0,47.0,0.0,3.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
4,6.0,1.0,114.0,600.0,97.0,16.17,4901.0,100.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,36.0,14.0,193.0,2461.0,201.0,8.17,19106.0,18.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
798,37.0,12.0,173.0,746.0,101.0,13.54,6799.0,39.0,2.0,3.0,9.0,0.0,1.0,0.0,0.0,0.0,0.0
799,38.0,5.0,97.0,649.0,150.0,23.11,5792.0,51.0,3.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0
800,39.0,3.0,7.0,333.0,14.0,4.20,2440.0,4.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0


In [67]:
#visilastion_graphe(normalized_min_max, Y)
ziped, predicts = modelss(X_chi, Y, X_Test, Y_Test)
# ziped_smote, predicts_smote = modelss(X_smote, Y_smote, X_val, Y_val)
# ziped_adasyn, predicts_smote = modelss(X_adasyn, Y_adasyn, X_val, Y_val)
#total_score = Random_Forest_desicion_of_depths(X,Y)
#total_score2 = Random_Forest_desicion_of_trees(X,Y)
#score, feature_importance = Random_Forest(X,Y)
# all_models_adasyn = pd.DataFrame(ziped_adasyn,  columns =['MODEL', 'MAE with adasyn', 'STD with adasyn', 'Val score with adasyn'])
# all_models_smote = pd.DataFrame(ziped_smote,  columns =['MODEL', 'MAE with smote', 'STD with smote', 'Val score with smote'])
#all_models = pd.DataFrame(ziped,  columns =['MODEL', 'MAE', 'STD', 'Val score'])
all_models = pd.DataFrame(ziped,  columns =['MODEL', 'MAE', 'STD'])


#feature_importance = pd.DataFrame(feature_importance,  columns =['feature name', 'importance'])
#pd.concat([all_models, all_models_smote, all_models_adasyn], axis=1)
all_models

,MODEL,MAE,STD
0,Random Forest,0.746862,0.052327
1,Linear Regression,0.760002,0.037592
2,Common - Stupid,0.794259,0.008436
3,XGB,0.796386,0.063774
4,SVM,0.821787,0.009757
5,Decision Tree,0.838086,0.098874
6,Mean - Stupid,0.866809,0.006216
7,KNN,0.874900,0.045038
